In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import f1_score, accuracy_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.initializers import he_uniform
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import skopt
from skopt import gbrt_minimize, gp_minimize
from skopt.utils import use_named_args
from skopt.space import Real, Categorical, Integer

C:\Users\tdtap\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
from learningratefinder import LearningRateFinder
from clr_callback import CyclicLR

In [3]:
# Set file paths for train and predict datasets
train_file = 'DataSet/Data_Train.xlsx'
predict_file = 'DataSet/Data_Test.xlsx'

In [4]:
# Extract train/predict data from spreadsheet into pandas dataframes
train_df = pd.read_excel(train_file)
predict_df = pd.read_excel(predict_file)

In [5]:
# Get numeric value from "Delivery_Time" field in train data
train_df['Del_Time'] = train_df['Delivery_Time'].apply(lambda x: pd.to_numeric(x.split('minutes')[0].strip()))
train_df['Del_Time_enc'] = train_df['Del_Time'].factorize()[0]
train_df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,Del_Time,Del_Time_enc
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200,50,3.5,12,4,30 minutes,30,0
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",100,50,3.5,11,4,30 minutes,30,0
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",150,50,3.6,99,30,65 minutes,65,1
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",250,99,3.7,176,95,30 minutes,30,0
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",200,99,3.2,521,235,65 minutes,65,1


In [6]:
train_df.groupby(['Del_Time', 'Del_Time_enc']).size().reset_index().rename(columns={0:'count'})

,Del_Time,Del_Time_enc,count
0,10,3,4
1,20,4,20
2,30,0,7406
3,45,2,2665
4,65,1,923
5,80,6,14
6,120,5,62


In [7]:
# Extract "Del_Time" field from train_df into NumPy array
train_y = np.array([train_df['Del_Time_enc'].values]).T
train_df.drop(['Del_Time', 'Delivery_Time', 'Del_Time_enc'], inplace=True, axis=1)
print("train_y: {}".format(train_y.shape))
print("Sample train_y data: \n{}".format(train_y[0:10,:]))

train_y: (11094, 1)
Sample train_y data: 
[[0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [0]
 [2]
 [0]
 [1]]


In [8]:
# Combine the train and predict dataframes
combined_df = train_df.append(predict_df, sort=False, ignore_index=True)
print("combined_df: {}".format(combined_df.shape))

combined_df: (13868, 8)


In [9]:
# Read location analysis data
loc_analysis_df = pd.read_excel("Location_Analysis.xlsx")
loc_analysis_df.head()

,Location,State,City,Area
0,"FTI College, Law College Road, Pune",Maharashtra,Pune,"FTI College, Law College Road"
1,"Sector 3, Marathalli",Karnataka,Bangalore,"Sector 3, Marathalli"
2,Mumbai Central,Maharashtra,Mumbai,Mumbai Central
3,"Sector 1, Noida",New Delhi,Noida,Sector 1
4,"Rmz Centennial, I Gate, Whitefield",Karnataka,Bangalore,"Rmz Centennial, I Gate, Whitefield"


In [10]:
# Derive "State", "City" and "Area" based on "Location" field
combined_df['State'] = combined_df['Location'].apply(lambda x: loc_analysis_df[loc_analysis_df['Location'] == x]['State'].max(axis=0))
combined_df['City'] = combined_df['Location'].apply(lambda x: loc_analysis_df[loc_analysis_df['Location'] == x]['City'].max(axis=0))
combined_df['Area'] = combined_df['Location'].apply(lambda x: loc_analysis_df[loc_analysis_df['Location'] == x]['Area'].max(axis=0))
combined_df.drop(['Location'], inplace=True, axis=1)
print("combined_df: {}".format(combined_df.shape))

combined_df: (13868, 10)


In [11]:
# Convert "State" field to one-hot encoding
dummy_df = pd.get_dummies(combined_df['State'], prefix='State')
combined_df = pd.concat([combined_df, dummy_df], axis=1)
combined_df.drop(['State'], inplace=True, axis=1)
print("combined_df: {}".format(combined_df.shape))

combined_df: (13868, 14)


In [12]:
# Convert "City" field to one-hot encoding
dummy_df = pd.get_dummies(combined_df['City'], prefix='City')
combined_df = pd.concat([combined_df, dummy_df], axis=1)
combined_df.drop(['City'], inplace=True, axis=1)
print("combined_df: {}".format(combined_df.shape))

combined_df: (13868, 24)


In [13]:
# Column encoding for "Area" field
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, ngram_range=(1, 1), stop_words='english', max_features=10000)
features = tfidf.fit_transform(combined_df.Area).toarray()
features_df = pd.DataFrame(features, columns=tfidf.get_feature_names())
combined_df = pd.merge(combined_df, features_df, left_index=True, right_index=True)
combined_df.drop(['Area'], inplace=True, axis=1)
print("combined_df: {}".format(combined_df.shape))

combined_df: (13868, 94)


In [14]:
# Convert "Cuisines" field into categorical encoded individual fields
combined_df['Cuisine_List'] = combined_df['Cuisines'].apply(lambda x: x.split(', '))
combined_df = combined_df.drop('Cuisine_List', 1).join(combined_df.Cuisine_List.str.join('|').str.get_dummies())
combined_df.drop(['Cuisines'], inplace=True, axis=1)
print("combined_df: {}".format(combined_df.shape))

combined_df: (13868, 194)


In [15]:
# Fill invalid string value in "Average_Cost" field with mean value
combined_df['Avg_Cost'] = combined_df['Average_Cost'].apply(lambda x: np.nan if x == 'for' else x)
mean_cost = combined_df['Avg_Cost'].mean()
combined_df['Avg_Cost'] = combined_df['Avg_Cost'].fillna(mean_cost)
combined_df.drop(['Average_Cost'], inplace=True, axis=1)
print("combined_df: {}".format(combined_df.shape))

combined_df: (13868, 194)


In [16]:
# Fill invalid string value in "Rating" field based on below rule:
# 1) If ['NEW','Opening Soon','Temporarily Closed'] then 0
# 2) If '-', then NaN
combined_df['modified_rating'] = combined_df['Rating'].apply(lambda x: np.nan if x == '-' else (0 if x in ['NEW','Opening Soon','Temporarily Closed'] else pd.to_numeric(x)))
mean_rating = combined_df['modified_rating'].mean()
combined_df['modified_rating'] = combined_df['modified_rating'].fillna(mean_rating)
combined_df.drop(['Rating'], inplace=True, axis=1)
print("combined_df: {}".format(combined_df.shape))

combined_df: (13868, 194)


In [17]:
# Fill invalid string value in "Votes" field with mean value
combined_df['modified_votes'] = combined_df['Votes'].apply(lambda x: np.nan if x == '-' else pd.to_numeric(x))
mean_cost = combined_df['modified_votes'].mean()
combined_df['modified_votes'] = combined_df['modified_votes'].fillna(mean_cost)
combined_df.drop(['Votes'], inplace=True, axis=1)
print("combined_df: {}".format(combined_df.shape))

combined_df: (13868, 194)


In [18]:
# Fill invalid string value in "Reviews" field with mean value
combined_df['modified_reviews'] = combined_df['Reviews'].apply(lambda x: np.nan if x == '-' else pd.to_numeric(x))
mean_cost = combined_df['modified_reviews'].mean()
combined_df['modified_reviews'] = combined_df['modified_reviews'].fillna(mean_cost)
combined_df.drop(['Reviews'], inplace=True, axis=1)
print("combined_df: {}".format(combined_df.shape))

combined_df: (13868, 194)


In [19]:
combined_df.drop(['Restaurant'], inplace=True, axis=1)
print("combined_df: {}".format(combined_df.shape))

combined_df: (13868, 193)


In [20]:
# Check if any column has NaN value in dataframe
print("Column with NaN value: {}".format(combined_df.columns[combined_df.isnull().any()].tolist()))

Column with NaN value: []


In [21]:
# Segregate combined_df into train/predict datasets
train_x = combined_df[:11094]
predict_x = combined_df[11094:]

In [22]:
print(train_x.shape)
print(predict_x.shape)

(11094, 193)
(2774, 193)


In [23]:
# Scale the train_x/predict_x datasets
scaler_x = RobustScaler().fit(train_x)
train_x = scaler_x.transform(train_x)
predict_x = scaler_x.transform(predict_x)

In [24]:
# Split training data into train/validation/test datasets
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=1)
for train_index, validation_index in sss.split(train_x, train_y):
    train_x, validation_x = train_x[train_index], train_x[validation_index]
    train_y, validation_y = train_y[train_index], train_y[validation_index]

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1)
for train_index, test_index in sss.split(train_x, train_y):
    train_x, test_x = train_x[train_index], train_x[test_index]
    train_y, test_y = train_y[train_index], train_y[test_index]

In [25]:
print("------------------------- Training Dataset -------------------------")
print("train_x shape: {}".format(train_x.shape))
print("train_y shape: {}".format(train_y.shape))

print("\n------------------------- Validation Dataset -------------------------")
print("validation_x shape: {}".format(validation_x.shape))
print("validation_y shape: {}".format(validation_y.shape))

print("\n------------------------- Test Dataset -------------------------")
print("test_x shape: {}".format(test_x.shape))
print("test_y shape: {}".format(test_y.shape))

------------------------- Training Dataset -------------------------
train_x shape: (7543, 193)
train_y shape: (7543, 1)

------------------------- Validation Dataset -------------------------
validation_x shape: (1665, 193)
validation_y shape: (1665, 1)

------------------------- Test Dataset -------------------------
test_x shape: (1886, 193)
test_y shape: (1886, 1)


In [26]:
# Handling class imbalance
sm = SMOTE(k_neighbors=1)
sm_x, sm_y = sm.fit_sample(train_x, train_y.ravel())
train_x = sm_x
train_y = np.array([sm_y]).T
print("Class balancing done.")
print("train_x shape: {}".format(train_x.shape))
print("train_y shape: {}".format(train_y.shape))

Class balancing done.
train_x shape: (35252, 193)
train_y shape: (35252, 1)


In [27]:
# Convert train_y/validation_y/test_y into one-hot encode matrix
train_y = tf.keras.utils.to_categorical(train_y, 7)
validation_y = tf.keras.utils.to_categorical(validation_y, 7)
test_y = tf.keras.utils.to_categorical(test_y, 7)

In [28]:
print("------------------------- Training Dataset -------------------------")
print("train_x shape: {}".format(train_x.shape))
print("train_y shape: {}".format(train_y.shape))

print("\n------------------------- Validation Dataset -------------------------")
print("validation_x shape: {}".format(validation_x.shape))
print("validation_y shape: {}".format(validation_y.shape))

print("\n------------------------- Test Dataset -------------------------")
print("test_x shape: {}".format(test_x.shape))
print("test_y shape: {}".format(test_y.shape))

------------------------- Training Dataset -------------------------
train_x shape: (35252, 193)
train_y shape: (35252, 7)

------------------------- Validation Dataset -------------------------
validation_x shape: (1665, 193)
validation_y shape: (1665, 7)

------------------------- Test Dataset -------------------------
test_x shape: (1886, 193)
test_y shape: (1886, 7)


In [29]:
dim_learning_rate = Real(low=1e-4, high=1e-2, prior='log-uniform', name='learning_rate')
dim_num_dense_layers = Integer(low=1, high=5, name='num_dense_layers')
dim_num_input_nodes = Integer(low=16, high=128, name='num_input_nodes')
dim_num_dense_nodes_FC2 = Integer(low=16, high=128, name='num_dense_nodes_FC2')
dim_num_dense_nodes_FC3 = Integer(low=16, high=128, name='num_dense_nodes_FC3')
dim_num_dense_nodes_FC4 = Integer(low=8, high=128, name='num_dense_nodes_FC4')
dim_num_dense_nodes_FC5 = Integer(low=8, high=128, name='num_dense_nodes_FC5')
dim_adam_decay = Real(low=1e-6, high=1e-2, name="adam_decay")

dimensions = [dim_learning_rate,
              dim_num_dense_layers,
              dim_num_input_nodes,
              dim_num_dense_nodes_FC2,
              dim_num_dense_nodes_FC3,
              dim_num_dense_nodes_FC4,
              dim_num_dense_nodes_FC5,
              dim_adam_decay]

default_parameters = [1e-3, 1, 128, 64, 64, 32, 32, 1e-3]

input_shape = train_x.shape[1]

In [30]:
def create_model(learning_rate, num_dense_layers, num_input_nodes, adam_decay,
                num_dense_nodes_FC2, num_dense_nodes_FC3, num_dense_nodes_FC4, num_dense_nodes_FC5):
    
    # Input Layer
    x_input = Input(shape=(input_shape, ), name='INPUT')
    
    # Fully-connected Layer 1
    x = Dense(units=num_input_nodes, name='FC-1', activation='relu', kernel_initializer=he_uniform(seed=1), kernel_regularizer=l2(0.1))(x_input)
    x = BatchNormalization(name='BN_FC-1')(x)
    x = Dropout(rate=0.5, name='DROPOUT_FC-1')(x)
    
    # Fully-connected Layer 2
    x = Dense(units=num_dense_nodes_FC2, name='FC-2', activation='relu', kernel_initializer=he_uniform(seed=1), kernel_regularizer=l2(0.1))(x)
    x = BatchNormalization(name='BN_FC-2')(x)
    x = Dropout(rate=0.5, name='DROPOUT_FC-2')(x)
    
    # Fully-connected Layer 3
    x = Dense(units=num_dense_nodes_FC3, name='FC-3', activation='relu', kernel_initializer=he_uniform(seed=1), kernel_regularizer=l2(0.1))(x)
    x = BatchNormalization(name='BN_FC-3')(x)
    x = Dropout(rate=0.5, name='DROPOUT_FC-3')(x)
    
    # Fully-connected Layer 4
    x = Dense(units=num_dense_nodes_FC4, name='FC-4', activation='relu', kernel_initializer=he_uniform(seed=1), kernel_regularizer=l2(0.1))(x)
    x = BatchNormalization(name='BN_FC-4')(x)
    x = Dropout(rate=0.5, name='DROPOUT_FC-4')(x)
    
    # Fully-connected Layer 5
    x = Dense(units=num_dense_nodes_FC5, name='FC-5', activation='relu', kernel_initializer=he_uniform(seed=1), kernel_regularizer=l2(0.1))(x)
    x = BatchNormalization(name='BN_FC-5')(x)
    x = Dropout(rate=0.5, name='DROPOUT_FC-5')(x)
    
    # Output Layer
    x = Dense(units=7, activation='softmax', name='OUTPUT', kernel_initializer=he_uniform(seed=1))(x)

    # Create Keras Model instance
    model = Model(inputs=x_input, outputs=x, name='Model')
    
    adam = Adam(lr=learning_rate, decay= adam_decay)
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])

    return model

In [31]:
@use_named_args(dimensions=dimensions)
def fitness(learning_rate, num_dense_layers, num_input_nodes, adam_decay,
            num_dense_nodes_FC2, num_dense_nodes_FC3, num_dense_nodes_FC4, num_dense_nodes_FC5):

    model = create_model(learning_rate=learning_rate,
                         num_dense_layers=num_dense_layers,
                         num_input_nodes=num_input_nodes,
                         adam_decay=adam_decay,
                         num_dense_nodes_FC2=num_dense_nodes_FC2,
                         num_dense_nodes_FC3=num_dense_nodes_FC3,
                         num_dense_nodes_FC4=num_dense_nodes_FC4,
                         num_dense_nodes_FC5=num_dense_nodes_FC5,
                        )
    
    blackbox = model.fit(x=train_x, y=train_y, 
                         epochs=30, batch_size=128, 
                         validation_data=(validation_x, validation_y))

    accuracy = blackbox.history['val_accuracy'][-1]

    # Print the classification accuracy.
    print()
    print("Accuracy: {0:.2%}".format(accuracy))
    print()

    
    # Delete the Keras model with these hyper-parameters from memory.
    del model
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    
    return -accuracy

In [32]:
gp_result = gp_minimize(func=fitness, dimensions=dimensions,
                        n_calls=70, noise= 0.01, n_jobs=-1,
                        kappa = 5, x0=default_parameters)

Train on 35252 samples, validate on 1665 samples
Epoch 1/30
35252/35252 [==============================] - 11s 299us/sample - loss: 23.9127 - accuracy: 0.2375 - val_loss: 7.1181 - val_accuracy: 0.0126
Epoch 2/30
35252/35252 [==============================] - 3s 96us/sample - loss: 3.6894 - accuracy: 0.4694 - val_loss: 2.2913 - val_accuracy: 0.2408
Epoch 3/30
35252/35252 [==============================] - 3s 90us/sample - loss: 1.3878 - accuracy: 0.6402 - val_loss: 1.5904 - val_accuracy: 0.2498
Epoch 4/30
35252/35252 [==============================] - 3s 93us/sample - loss: 0.9863 - accuracy: 0.6944 - val_loss: 1.3636 - val_accuracy: 0.4787
Epoch 5/30
35252/35252 [==============================] - 3s 92us/sample - loss: 0.8839 - accuracy: 0.7148 - val_loss: 1.4137 - val_accuracy: 0.4306
Epoch 6/30
35252/35252 [==============================] - 3s 91us/sample - loss: 0.8194 - accuracy: 0.7305 - val_loss: 1.2639 - val_accuracy: 0.5357
Epoch 7/30
35252/35252 [==============================

Epoch 25/30
35252/35252 [==============================] - 4s 115us/sample - loss: 0.6259 - accuracy: 0.7644 - val_loss: 1.1246 - val_accuracy: 0.5562
Epoch 26/30
35252/35252 [==============================] - 4s 117us/sample - loss: 0.6248 - accuracy: 0.7671 - val_loss: 1.1162 - val_accuracy: 0.5586
Epoch 27/30
35252/35252 [==============================] - 4s 115us/sample - loss: 0.6131 - accuracy: 0.7665 - val_loss: 1.0930 - val_accuracy: 0.5574
Epoch 28/30
35252/35252 [==============================] - 4s 117us/sample - loss: 0.6207 - accuracy: 0.7646 - val_loss: 1.1162 - val_accuracy: 0.5550
Epoch 29/30
35252/35252 [==============================] - 4s 110us/sample - loss: 0.6119 - accuracy: 0.7698 - val_loss: 1.1422 - val_accuracy: 0.5285
Epoch 30/30
35252/35252 [==============================] - 4s 110us/sample - loss: 0.6047 - accuracy: 0.7699 - val_loss: 1.0997 - val_accuracy: 0.5586

Accuracy: 55.86%

Train on 35252 samples, validate on 1665 samples
Epoch 1/30
35252/35252 [==

Epoch 18/30
35252/35252 [==============================] - 4s 103us/sample - loss: 0.5705 - accuracy: 0.7817 - val_loss: 1.0971 - val_accuracy: 0.5357
Epoch 19/30
35252/35252 [==============================] - 4s 102us/sample - loss: 0.5652 - accuracy: 0.7855 - val_loss: 1.0754 - val_accuracy: 0.5628
Epoch 20/30
35252/35252 [==============================] - 4s 100us/sample - loss: 0.5516 - accuracy: 0.7886 - val_loss: 1.0842 - val_accuracy: 0.5550
Epoch 21/30
35252/35252 [==============================] - 4s 102us/sample - loss: 0.5455 - accuracy: 0.7889 - val_loss: 1.0718 - val_accuracy: 0.5640
Epoch 22/30
35252/35252 [==============================] - 4s 103us/sample - loss: 0.5460 - accuracy: 0.7901 - val_loss: 1.0698 - val_accuracy: 0.5646
Epoch 23/30
35252/35252 [==============================] - 4s 104us/sample - loss: 0.5393 - accuracy: 0.7916 - val_loss: 1.0694 - val_accuracy: 0.5580
Epoch 24/30
35252/35252 [==============================] - 4s 103us/sample - loss: 0.5331 - ac

Epoch 4/30
35252/35252 [==============================] - 4s 113us/sample - loss: 1.3674 - accuracy: 0.6805 - val_loss: 1.4548 - val_accuracy: 0.5411
Epoch 5/30
35252/35252 [==============================] - 4s 117us/sample - loss: 0.9645 - accuracy: 0.7157 - val_loss: 1.4049 - val_accuracy: 0.4462
Epoch 6/30
35252/35252 [==============================] - 4s 115us/sample - loss: 0.8531 - accuracy: 0.7312 - val_loss: 1.3810 - val_accuracy: 0.4192
Epoch 7/30
35252/35252 [==============================] - 4s 118us/sample - loss: 0.7990 - accuracy: 0.7369 - val_loss: 1.2462 - val_accuracy: 0.4979
Epoch 8/30
35252/35252 [==============================] - 4s 114us/sample - loss: 0.7796 - accuracy: 0.7401 - val_loss: 1.2257 - val_accuracy: 0.5459
Epoch 9/30
35252/35252 [==============================] - 4s 115us/sample - loss: 0.7600 - accuracy: 0.7473 - val_loss: 1.2964 - val_accuracy: 0.4673
Epoch 10/30
35252/35252 [==============================] - 4s 116us/sample - loss: 0.7441 - accuracy

35252/35252 [==============================] - 3s 88us/sample - loss: 17.9195 - accuracy: 0.1899 - val_loss: 17.7404 - val_accuracy: 0.0426
Epoch 21/30
35252/35252 [==============================] - 3s 93us/sample - loss: 17.6750 - accuracy: 0.1924 - val_loss: 17.5113 - val_accuracy: 0.0426
Epoch 22/30
35252/35252 [==============================] - 3s 89us/sample - loss: 17.4363 - accuracy: 0.1943 - val_loss: 17.2828 - val_accuracy: 0.0462
Epoch 23/30
35252/35252 [==============================] - 3s 90us/sample - loss: 17.2300 - accuracy: 0.1910 - val_loss: 17.0826 - val_accuracy: 0.0432
Epoch 24/30
35252/35252 [==============================] - 3s 89us/sample - loss: 17.0194 - accuracy: 0.1940 - val_loss: 16.8751 - val_accuracy: 0.0462
Epoch 25/30
35252/35252 [==============================] - 3s 89us/sample - loss: 16.8233 - accuracy: 0.1945 - val_loss: 16.6964 - val_accuracy: 0.0462
Epoch 26/30
35252/35252 [==============================] - 3s 88us/sample - loss: 16.6485 - accuracy

...

Epoch 22/30
35252/35252 [==============================] - 2s 68us/sample - loss: 0.7395 - accuracy: 0.7011 - val_loss: 1.1276 - val_accuracy: 0.5417
Epoch 23/30
35252/35252 [==============================] - 2s 67us/sample - loss: 0.7301 - accuracy: 0.7048 - val_loss: 1.1338 - val_accuracy: 0.5153
Epoch 24/30
35252/35252 [==============================] - 3s 72us/sample - loss: 0.7288 - accuracy: 0.7055 - val_loss: 1.1557 - val_accuracy: 0.5261
Epoch 25/30
35252/35252 [==============================] - 2s 70us/sample - loss: 0.7208 - accuracy: 0.7106 - val_loss: 1.1356 - val_accuracy: 0.5249
Epoch 26/30
35252/35252 [==============================] - 2s 70us/sample - loss: 0.7179 - accuracy: 0.7067 - val_loss: 1.1270 - val_accuracy: 0.5267
Epoch 27/30
35252/35252 [==============================] - 2s 69us/sample - loss: 0.7124 - accuracy: 0.7080 - val_loss: 1.1490 - val_accuracy: 0.5171
Epoch 28/30
35252/35252 [==============================] - 2s 68us/sample - loss: 0.7124 - accuracy:

Epoch 16/30
35252/35252 [==============================] - 3s 75us/sample - loss: 0.7340 - accuracy: 0.7246 - val_loss: 1.1075 - val_accuracy: 0.5634
Epoch 17/30
35252/35252 [==============================] - 3s 76us/sample - loss: 0.7194 - accuracy: 0.7250 - val_loss: 1.1299 - val_accuracy: 0.5459
Epoch 18/30
35252/35252 [==============================] - 3s 76us/sample - loss: 0.7092 - accuracy: 0.7297 - val_loss: 1.1064 - val_accuracy: 0.5712
Epoch 19/30
35252/35252 [==============================] - 3s 74us/sample - loss: 0.7092 - accuracy: 0.7283 - val_loss: 1.0751 - val_accuracy: 0.5832
Epoch 20/30
35252/35252 [==============================] - 3s 76us/sample - loss: 0.7023 - accuracy: 0.7262 - val_loss: 1.0998 - val_accuracy: 0.5417
Epoch 21/30
35252/35252 [==============================] - 3s 76us/sample - loss: 0.6907 - accuracy: 0.7341 - val_loss: 1.0728 - val_accuracy: 0.5634
Epoch 22/30
35252/35252 [==============================] - 3s 75us/sample - loss: 0.6915 - accuracy:

Epoch 10/30
35252/35252 [==============================] - 2s 67us/sample - loss: 1.1610 - accuracy: 0.5358 - val_loss: 1.3810 - val_accuracy: 0.6138
Epoch 11/30
35252/35252 [==============================] - 2s 67us/sample - loss: 1.1185 - accuracy: 0.5429 - val_loss: 1.3701 - val_accuracy: 0.5832
Epoch 12/30
35252/35252 [==============================] - 2s 69us/sample - loss: 1.1027 - accuracy: 0.5521 - val_loss: 1.3840 - val_accuracy: 0.4270
Epoch 13/30
35252/35252 [==============================] - 2s 70us/sample - loss: 1.0913 - accuracy: 0.5518 - val_loss: 1.3178 - val_accuracy: 0.6114
Epoch 14/30
35252/35252 [==============================] - 2s 69us/sample - loss: 1.0767 - accuracy: 0.5642 - val_loss: 1.3126 - val_accuracy: 0.5922
Epoch 15/30
35252/35252 [==============================] - 2s 69us/sample - loss: 1.0434 - accuracy: 0.5654 - val_loss: 1.3370 - val_accuracy: 0.5868
Epoch 16/30
35252/35252 [==============================] - 3s 71us/sample - loss: 1.0479 - accuracy:

Epoch 4/30
35252/35252 [==============================] - 3s 80us/sample - loss: 13.2286 - accuracy: 0.3831 - val_loss: 10.3953 - val_accuracy: 0.1417
Epoch 5/30
35252/35252 [==============================] - 3s 78us/sample - loss: 8.2432 - accuracy: 0.4892 - val_loss: 6.7062 - val_accuracy: 0.4751
Epoch 6/30
35252/35252 [==============================] - 3s 78us/sample - loss: 5.2946 - accuracy: 0.5846 - val_loss: 4.5533 - val_accuracy: 0.5129
Epoch 7/30
35252/35252 [==============================] - 3s 79us/sample - loss: 3.5110 - accuracy: 0.6503 - val_loss: 3.1923 - val_accuracy: 0.6456
Epoch 8/30
35252/35252 [==============================] - 3s 79us/sample - loss: 2.3888 - accuracy: 0.6868 - val_loss: 2.4812 - val_accuracy: 0.3922
Epoch 9/30
35252/35252 [==============================] - 3s 78us/sample - loss: 1.7167 - accuracy: 0.6980 - val_loss: 1.9648 - val_accuracy: 0.4781
Epoch 10/30
35252/35252 [==============================] - 3s 79us/sample - loss: 1.3053 - accuracy: 0.7

35252/35252 [==============================] - 3s 85us/sample - loss: 0.6860 - accuracy: 0.7623 - val_loss: 1.2637 - val_accuracy: 0.5273
Epoch 22/30
35252/35252 [==============================] - 3s 86us/sample - loss: 0.6750 - accuracy: 0.7613 - val_loss: 1.1874 - val_accuracy: 0.5483
Epoch 23/30
35252/35252 [==============================] - 3s 86us/sample - loss: 0.6618 - accuracy: 0.7649 - val_loss: 1.1658 - val_accuracy: 0.5574
Epoch 24/30
35252/35252 [==============================] - 3s 87us/sample - loss: 0.6673 - accuracy: 0.7638 - val_loss: 1.2115 - val_accuracy: 0.5147
Epoch 25/30
35252/35252 [==============================] - 3s 88us/sample - loss: 0.6563 - accuracy: 0.7637 - val_loss: 1.2033 - val_accuracy: 0.5351
Epoch 26/30
35252/35252 [==============================] - 3s 86us/sample - loss: 0.6512 - accuracy: 0.7661 - val_loss: 1.2090 - val_accuracy: 0.5423
Epoch 27/30
35252/35252 [==============================] - 3s 85us/sample - loss: 0.6453 - accuracy: 0.7685 - va

In [33]:
res = pd.concat([pd.DataFrame(gp_result.x_iters, columns = ["learning rate","hidden layers",
                                                      "input layer nodes","hidden layer nodes FC2",
                                                      "hidden layer nodes FC3","hidden layer nodes FC4",
                                                      "hidden layer nodes FC5","adam learning rate decay"]),
(pd.Series(gp_result.func_vals*-100, name="accuracy"))], axis=1)

In [34]:
res.head()

,learning rate,hidden layers,input layer nodes,hidden layer nodes FC2,hidden layer nodes FC3,hidden layer nodes FC4,hidden layer nodes FC5,adam learning rate decay,accuracy
0,0.001000,1,128,64,64,32,32,0.001000,55.555557
1,0.006852,5,110,92,100,69,17,0.007759,55.855858
2,0.000147,5,48,110,103,36,23,0.004137,7.327327
3,0.000652,1,84,99,59,42,67,0.001777,56.396397
4,0.004903,2,23,125,105,100,44,0.006332,53.153152


In [35]:
res.to_excel("hyperparam_search.xlsx", index=False)

In [40]:
gp_result.x

[0.0016434051740089192, 1, 112, 102, 6, 72, 128, 0.0005188679603436311]